# 3. Dimensionality reduction

This tutorial on reducing the dimension of the scattering coefficients using an
independent component analysis. In this Jupyter notebook, we will walk through
the process of extracting the most relevant features in order to use them later
for clustering. We here follow again the indications in Steinmann et al.
([2021](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2021JB022455))
but we advise that many other methods for reducing the dimensions may be
relevant for other datasets. 

Made in 2022 by René Steinmann and Léonard Seydoux.

This notebook uses the __matplotlib__ and __scikit-learn__ library, please run the cell below if the packages are not installed.

In [ ]:
import pickle

from matplotlib import pyplot as plt
import numpy as np
from sklearn.decomposition import FastICA

plt.rcParams["date.converter"] = "concise"

## Load scattering coefficients

First, we load the scattering coefficients and reshape them for any [dimensionality reduction](https://scikit-learn.org/stable/modules/unsupervised_reduction.html) model (here `FastICA`) of the `scikit-learn` package. The shape of the scattering coefficients are given by the following tuples:

- Order 1: `(n_times, n_channel, octaves[0] * resolution[0])`
- Order 2: `(n_times, n_channel, octaves[0] * octaves[1] * resolution[0]  *
  resolution[1])`
- ...
- Order n: `(n_times, n_channel, np.prod(octaves) * np.prod(resolution))`

We then need to collect the all-order scattering coefficients into a
two-dimensional matrix for use with Scikit Learn. 

> Note that the optimal way to load the scattering coefficients is to use `xarray` 

In [ ]:
# Load data from file
with np.load("../example/scattering_coefficients.npz", allow_pickle=True) as data:
    order_1 = data["order_1"]
    order_2 = data["order_2"]
    times = data["times"]

# Reshape and stack scattering coefficients of all orders
order_1 = order_1.reshape(order_1.shape[0], -1)
order_2 = order_2.reshape(order_2.shape[0], -1)
scattering_coefficients = np.hstack((order_1, order_2))
# print(scattering_coefficients)

# transform into log
scattering_coefficients = np.log(scattering_coefficients)

# print info about shape
n_times, n_coeff = scattering_coefficients.shape
print("Collected {} samples of {} dimensions each.".format(n_times, n_coeff))

## Extract independant features

After loading and stacking the scattering coefficients into a matrix, we can now apply a dimentionality reduction algorithm. We here use the `FastICA` algorithm, but highly recommend to try other algorithms that will allow to proceed in the most adapted way to the data at hand. 

The `FastICA` algorithm looks for a matrix factorization with independent sources, and a mixing matrix. We need to inform the model about how many components (or features) we want to extract in the `n_components` keyword argument  below. The residual shape of the `features` matrix will be `(n_times, n_components)` instead of the initial scattering coefficients shape shown above.

In [ ]:
scattering_coefficients

In [ ]:
model = FastICA(n_components=10, whiten="unit-variance")
features = model.fit_transform(scattering_coefficients)

### Save the output

We here save the extracted features scattering coefficients as a npz-file, like the scattering coefficients in the previous notebook.

In [ ]:
# Save the features
np.savez(
    "../example/independent_components.npz",
    features=features,
    times=times,
)

# Save the dimension reduction model
with open("../example/dimension_model.pickle", "wb") as pickle_file:
    pickle.dump(
        model,
        pickle_file,
        protocol=pickle.HIGHEST_PROTOCOL,
    )

### Have a look at the features

This is a crucial step: do you see structure in the independent components, or do they seem all random? Do the structures correlate with the a priori information at your disposal or to identifiable signal structures? At this stage, it is important to carefully address those questions. The optimal solution may not be to use the `FastICA` model depending on the data at hand, although it can be adapated to many datasets. Playing with the number of components can also be an immportant way of creating structure in the output.

In [ ]:
# Normalize features for display
features_normalized = features / np.abs(features).max(axis=0)

# Figure instance
fig = plt.figure(dpi=200)
ax = plt.axes()

# Plot features
ax.plot(times, features_normalized + np.arange(features.shape[1]), rasterized=True)

# Labels
ax.set_ylabel("Feature index")
ax.set_xlabel("Date and time")

# Show
plt.show()